# Notebook 5: Clean Up Resources

Specify "Python 3" Kernel  and "Data Science" Image.

### Background

In this notebook, we will clean up the resources we provisioned during this workshop:

- SageMaker Feature Groups
- SageMaker Endpoints
- Amazon Kinesis Data Stream
- Amazon Kinesis Data Analytics application

### Imports

In [ ]:
from parameter_store import ParameterStore
from utils import *

### Session variables

In [ ]:
role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name
boto_session = boto3.Session()
kinesis_client = boto_session.client(service_name='kinesis', region_name=region)
kinesis_analytics_client = boto_session.client('kinesisanalytics')

ps = ParameterStore(verbose=False)
ps.set_namespace('feature-store-workshop')

Load variables from previous notebooks

In [ ]:
parameters = ps.read()

customers_feature_group_name = parameters['customers_feature_group_name']
products_feature_group_name = parameters['products_feature_group_name']
orders_feature_group_name = parameters['orders_feature_group_name']
click_stream_historical_feature_group_name = parameters['click_stream_historical_feature_group_name']
click_stream_feature_group_name = parameters['click_stream_feature_group_name']

cf_model_endpoint_name = parameters['cf_model_endpoint_name']
ranking_model_endpoint_name = parameters['ranking_model_endpoint_name']

kinesis_stream_name = parameters['kinesis_stream_name']
kinesis_analytics_application_name = parameters['kinesis_analytics_application_name']

### Delete feature groups

In [ ]:
feature_group_list = [customers_feature_group_name, products_feature_group_name,
                      orders_feature_group_name, click_stream_historical_feature_group_name,
                      click_stream_feature_group_name]

for feature_group in feature_group_list:
    print(f'Deleting feature group: {feature_group}')
    delete_feature_group(feature_group)

### Delete endpoints and endpoint configurations

In [ ]:
def clean_up_endpoint(endpoint_name):
    response = sagemaker_session.sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
    endpoint_config_name = response['EndpointConfigName']
    print(f'Deleting endpoint: {endpoint_name}')
    print(f'Deleting endpoint configuration : {endpoint_config_name}')
    sagemaker_session.sagemaker_client.delete_endpoint(EndpointName=endpoint_name)
    sagemaker_session.sagemaker_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)

In [ ]:
endpoint_list = [cf_model_endpoint_name, ranking_model_endpoint_name]

for endpoint in endpoint_list:
    clean_up_endpoint(endpoint)

### Delete Kinesis Data Stream

In [ ]:
kinesis_client.delete_stream(StreamName=kinesis_stream_name,
                             EnforceConsumerDeletion=True)

### Delete Kinesis Data Analytics application

In [ ]:
response = kinesis_analytics_client.describe_application(ApplicationName=kinesis_analytics_application_name)
create_ts = response['ApplicationDetail']['CreateTimestamp']
kinesis_analytics_client.delete_application(ApplicationName=kinesis_analytics_application_name, CreateTimestamp=create_ts)

Go back to Workshop Studio and click on "Next".